# Emsemble try

- NBLogistic
- NBSVM
- Logistic Regression
- Random Forest

.
.
.

In [82]:
import os
import sys

src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [92]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from nbextractor import NBExtractor
from nbmodels import NbLogisticClassifier
from sklearn.pipeline import Pipeline
import re
import string

# Data

In [113]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

## Data preparation

In [114]:
def tokenize(s):
    pattern = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
    return pattern.sub(r' \1 ', s).split()

In [115]:
train_data['comment_text'].fillna("unknown", inplace=True)
test_data['comment_text'].fillna("unknown", inplace=True)

In [17]:
X = train.comment_text.values
idx = np.arange(len(X))

In [18]:
X_train, X_test, idx_train, idx_test = train_test_split(X, idx, test_size=0.2)

### Without tfidf optimization

In [21]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
                        min_df=3, max_df=0.9, strip_accents='unicode',
                        use_idf=1, smooth_idf=1, sublinear_tf=1, stop_words='english')

In [27]:
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

# Good and old Logistic Regression

In [63]:
params = {
    'penalty': ['l1', 'l2'],
    'C': [1e-4 * (10**i) for i in range(0, 8)]
}

logisticclf = GridSearchCV(LogisticRegression(), params, n_jobs=5, verbose=True, scoring='roc_auc')

In [64]:
best_models = {}

probs = np.zeros(shape=(len(X_test), 6))
for i, col in enumerate(train.columns[2:]):
    print("Optimizing logistic for", col)
    y_train = train.loc[idx_train, col].values
    y_test = train.loc[idx_test, col].values
    logisticclf.fit(X_train_vec, y_train)
    best_models[col]['params'] = logisticclflogisticclf.best_params_
    best_models[col]['score'] = logisticclf.best_score_
    print('Best model:')
    print(best_models[col])
    y_pred = logisticclf.predict_proba(X_test_vec)[:,1]
    probs[:, i] = y_pred
    auc = roc_auc_score(y_test, y_pred)
    lg_loss = log_loss(y_test, y_pred)
    print("{} auc: {}".format(col, auc))
    print("{} log loss: {}".format(col, lg_loss))

Optimizing logistic for toxic
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=5)]: Done  48 out of  48 | elapsed:  1.1min finished


Best model:
{'cv': None, 'error_score': 'raise', 'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__max_iter': 100, 'estimator__multi_class': 'ovr', 'estimator__n_jobs': 1, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'liblinear', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'fit_params': None, 'iid': True, 'n_jobs': 5, 'param_grid': {'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': 'warn', 'scoring': 'roc_auc', 'verbose': True}
to

[Parallel(n_jobs=5)]: Done  48 out of  48 | elapsed:  1.1min finished


Best model:
{'cv': None, 'error_score': 'raise', 'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__max_iter': 100, 'estimator__multi_class': 'ovr', 'estimator__n_jobs': 1, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'liblinear', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'fit_params': None, 'iid': True, 'n_jobs': 5, 'param_grid': {'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': 'warn', 'scoring': 'roc_auc', 'verbose': True}
se

...

Best model:
{'cv': None, 'error_score': 'raise', 'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__max_iter': 100, 'estimator__multi_class': 'ovr', 'estimator__n_jobs': 1, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'liblinear', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'fit_params': None, 'iid': True, 'n_jobs': 5, 'param_grid': {'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': 'warn', 'scoring': 'roc_auc', 'verbose': True}
th

[Parallel(n_jobs=5)]: Done  48 out of  48 | elapsed:  1.1min finished


Best model:
{'cv': None, 'error_score': 'raise', 'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__max_iter': 100, 'estimator__multi_class': 'ovr', 'estimator__n_jobs': 1, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'liblinear', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'fit_params': None, 'iid': True, 'n_jobs': 5, 'param_grid': {'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': 'warn', 'scoring': 'roc_auc', 'verbose': True}
in

[Parallel(n_jobs=5)]: Done  48 out of  48 | elapsed:   59.7s finished


Best model:
{'cv': None, 'error_score': 'raise', 'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__max_iter': 100, 'estimator__multi_class': 'ovr', 'estimator__n_jobs': 1, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'liblinear', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'fit_params': None, 'iid': True, 'n_jobs': 5, 'param_grid': {'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': 'warn', 'scoring': 'roc_auc', 'verbose': True}
id

As you can see the default values for a simple logistic regression already perform very well


# Random Forest

In [75]:
random_forest_params = {}
random_forest_clf = GridSearchCV(RandomForestClassifier(), random_forest_params,n_jobs=5, scoring='roc_auc')

In [79]:
best_random_forest_models = {}

probs = np.zeros(shape=(len(X_test), 6))
for i, col in enumerate(train.columns[2:]):
    print("Optimizing random forest for", col)
    y_train = train.loc[idx_train, col].values
    y_test = train.loc[idx_test, col].values
    random_forest_clf.fit(X_train_vec, y_train)
    best_random_forest_models[col] = {}
    best_random_forest_models[col]['params'] = random_forest_clf.best_params_
    best_random_forest_models[col]['score'] = random_forest_clf.best_score_
    print('Best model:')
    print(best_random_forest_models[col])
    y_pred = random_forest_clf.predict_proba(X_test_vec)[:,1]
    probs[:, i] = y_pred
    auc = roc_auc_score(y_test, y_pred)
    lg_loss = log_loss(y_test, y_pred)
    print("{} auc: {}".format(col, auc))
    print("{} log loss: {}".format(col, lg_loss))

Optimizing random forest for toxic
Best model:
{'params': {}, 'score': 0.9194933897517927}
toxic auc: 0.9268155013134192
toxic log loss: 0.36850545462149936
Optimizing random forest for severe_toxic
Best model:
{'params': {}, 'score': 0.8516252913034689}
severe_toxic auc: 0.8493212611679346
severe_toxic log loss: 0.11109988884503577
Optimizing random forest for obscene


KeyboardInterrupt: 

# NB Logistic

In [93]:
NBLogistic = Pipeline([
        ('naive_bayes_extractor', NBExtractor()),
        ('logistic_regression', LogisticRegression(n_jobs=5, solver='saga'))
    ])
nblogistic_clf = NbLogisticClassifier()

In [122]:
probs_train = np.zeros(shape=(len(X_train), 6))
probs_test = np.zeros(shape=(len(X_test), 6))
probs_sub = np.zeros(shape=(len(test_data), 6))
probs_sub_train = np.zeros(shape=(len(X), 6))
X_sub_train = tfidf.transform(X)
X_sub = tfidf.transform(test_data.comment_text)
for i, col in enumerate(train.columns[2:]):
    print("Training NBLogistic for", col)
    y_train = train.loc[idx_train, col].values
    y_test = train.loc[idx_test, col].values
    
    nblog = NbLogisticClassifier()
    nblogistic_clf.fit(X_train_vec, y_train)
    
    y_pred_train = nblogistic_clf.predict_proba(X_train_vec)[:,1]
    probs_train[:, i] = y_pred_train
    
    y_pred_test = nblogistic_clf.predict_proba(X_test_vec)[:,1]
    probs_test[:, i] = y_pred_test
    
    y_pred_sub = nblogistic_clf.predict_proba(X_sub)[:,1]
    probs_sub[:, i] = y_pred_sub
    
    y_pred_sub_train = nblogistic_clf.predict_proba(X_sub_train)[:,1]
    probs_sub_train[:, i] = y_pred_sub_train
    
    auc = roc_auc_score(y_test, y_pred_test)
    lg_loss = log_loss(y_test, y_pred_test)
    print("{} auc: {}".format(col, auc))
    print("{} log loss: {}".format(col, lg_loss))

Training NBLogistic for toxic
toxic auc: 0.9771727698300446
toxic log loss: 0.10592998557323856
Training NBLogistic for severe_toxic
severe_toxic auc: 0.9852819909779231
severe_toxic log loss: 0.02954962232495569
Training NBLogistic for obscene
obscene auc: 0.9901955034213097
obscene log loss: 0.05557560689789707
Training NBLogistic for threat
threat auc: 0.9848477359688248
threat log loss: 0.011327131151664654
Training NBLogistic for insult
insult auc: 0.9806088010448459
insult log loss: 0.07727378257892141
Training NBLogistic for identity_hate
identity_hate auc: 0.9669349853689663
identity_hate log loss: 0.02603729628843048


In [118]:
submission = pd.read_csv('../data/sample_submission.csv')
submission.loc[:, train.columns[2:]] = probs_sub

In [120]:
submission.to_csv('../submissions/nblogistic.csv', index=False)

In [125]:
features = pd.DataFrame(columns=submission.columns)
features['id'] = train_data.id

In [129]:
features.loc[:, features.columns[1:]] = probs_sub_train

In [131]:
features.to_csv('../data/features/features_nblogistic.csv')

# NB SVM

# Ensemble



In [ ]:
from mlextend.classifier import EnsembleVoteClassifier